In [26]:
import time,os,json
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torch.utils.data import Dataset
import easyocr

In [27]:
input_size = 256

data_transforms_train = torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(input_size),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_transforms_test = torchvision.transforms.Compose([
        torchvision.transforms.Resize(input_size),
        torchvision.transforms.CenterCrop(input_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [28]:
class ConTextDataset(Dataset):
    def __init__(self,dir_images_labels, dir_images, train = True, transform = None):
        self.dir_images        = dir_images
        self.dir_images_labels = dir_images_labels
        self.transform = transform

        if train:
            path = os.path.join(dir_images_labels,'train.txt')
        else:
            path = os.path.join(dir_images_labels,'test.txt')
        
        with open(path, 'r') as file:
            self.samples = [tuple(line.split()) for line in file]

    def __len__(self):
        return (len(self.samples))

    def __getitem__(self, idx):


        img_name = os.path.join(self.dir_images, self.samples[idx][0]+'.jpg')
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        target = torch.tensor(int(self.samples[idx][1]))

        return image, target


dir_images      = 'C:/Users/34644/Desktop/BUISNESS DETECTION/data/JPEGImages/'
dir_imgs_labels = 'C:/Users/34644/Desktop/BUISNESS DETECTION/data/'

train_Set = ConTextDataset(dir_imgs_labels, dir_images, train = True,  transform = data_transforms_train)
test_Set  = ConTextDataset(dir_imgs_labels, dir_images, train = False, transform = data_transforms_test)

In [31]:
reader = easyocr.Reader(['en'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


In [62]:

def obtain_ocr(train = True):
    ocr_file_path = 'C:/Users/34644/Desktop/BUISNESS DETECTION/data/ocr_train.txt'
    new_path_train = 'C:/Users/34644/Desktop/BUISNESS DETECTION/data/train.txt'

    dir_images      = 'C:/Users/34644/Desktop/BUISNESS DETECTION/data/JPEGImages/'
    with open(ocr_file_path, 'w') as ocr_file, open(new_path_train, 'r') as train_file:
        for line in train_file:
            img_name, label = line.split()
            file_path = os.path.join(dir_images, img_name + '.jpg')
            ocr_output = ''
            for text in reader.readtext(file_path):
                ocr_output += ' ' + text[1]
            ocr_file.write(ocr_output + '\n')

    
    

KeyboardInterrupt: 

In [47]:
img_name = os.path.join(dir_images, train_Set.samples[343][0]+'.jpg')
obj = reader.readtext(img_name)

In [60]:
ocr_output = ''
for text in obj:
    ocr_output += ' '+text[1]
print(ocr_output.split())

['BAKERY', '&', 'SHAVE', 'Ice', 'Waiola', 'Shave', '#ce"', 'OZEN', '4a0"', 'FNAG', 'Lt', 'Intt', 'WAIOLA']


In [ ]:
reader.readtext()

In [8]:
train_loader = torch.utils.data.DataLoader(train_Set, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_Set,  batch_size=64, shuffle=False)

In [22]:
#with open(path, 'r') as file:
#      images = []
#      labels = []
#      for line in file:
#          line_split = line.split()
#          try:
#              image_path = os.path.join(self.root_images, line_split[0] + '.jpg')
#              image = Image.open(image_path).convert('RGB')
#              if self.transform:
#                  image = self.transform(image)
#              label = int(line_split[1])
#              images.append(image)
#              labels.append(label)
#          except Exception as e:
#              print(f"Error loading image: {image_path} - {str(e)}")
#              continu#
#      self.images = torch.stack(images)  # Convert the list of images to a tensor
#      self.labels = torch.tensor(labels)  # Convert the list of labels to a tensor

RuntimeError: Couldn't open shared file mapping: <00000242C3348982>, error code: <1450>